In [40]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
import PIL
from keras import layers, optimizers
from keras.models import Model, model_from_json
from keras.layers import Input, Activation, Dense, Conv2D, Flatten, ZeroPadding2D, BatchNormalization
from keras.layers import MaxPooling2D, AveragePooling2D, Dropout, GlobalMaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array
from random import seed 
from random import randint


In [41]:
import cv2
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import keras.backend as K
K.set_image_data_format('channels_last')
from PIL import Image

In [42]:
directory = os.path.abspath("Transfer_Train")
print(directory)

C:\Users\Asus\Cat vs Dog Project\Transfer_Train


In [43]:
X = []
Y = []
i = 0
category = ['Cat', 'Dog', 'Chick']
for c in category:
    path = os.path.join(directory, c)
    for file in os.listdir(path):
        classes = category.index(c)
        Y.append(classes)
        img_path = os.path.join(path, file)
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        img_r = cv2.resize(img, (100, 100))
        X.append(img_r)
        
X_train = np.array(X)
Y_train = np.array(Y)
Y_train = to_categorical(Y_train)
cv2.imshow("New Image", X_train[35])
cv2.waitKey(0)
cv2.destroyAllWindows()

In [44]:
np.random.seed(1)
m = X_train.shape[0]
permutation = list(np.random.permutation(m))
X = X_train[permutation, :]
Y = Y_train[permutation, :]
X_train = X
Y_train = Y
cv2.imshow("new img", X_train[35])
print(Y_train[35])
cv2.waitKey(0)
cv2.destroyAllWindows()

[0. 1. 0.]


In [45]:
with open("ASIRA_model.json", "r") as json_file:
    loaded_model_json = json_file.read()
loaded_model = model_from_json(loaded_model_json)

In [46]:
loaded_model.load_weights("ASIRA_model_weights.h5")
print("Loaded")

Loaded


In [47]:
print(loaded_model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100, 100, 3)       0         
_________________________________________________________________
bn_0 (BatchNormalization)    (None, 100, 100, 3)       12        
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 108, 108, 3)       0         
_________________________________________________________________
conv_1 (Conv2D)              (None, 52, 52, 32)        2432      
_________________________________________________________________
bn_1 (BatchNormalization)    (None, 52, 52, 32)        128       
_________________________________________________________________
activation_1 (Activation)    (None, 52, 52, 32)        0         
_________________________________________________________________
mp_1 (MaxPooling2D)          (None, 26, 26, 32)        0         
__________

In [48]:
last_few_layers = 9
for l in loaded_model.layers[:-last_few_layers]:
    l.trainable = False
print(loaded_model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100, 100, 3)       0         
_________________________________________________________________
bn_0 (BatchNormalization)    (None, 100, 100, 3)       12        
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 108, 108, 3)       0         
_________________________________________________________________
conv_1 (Conv2D)              (None, 52, 52, 32)        2432      
_________________________________________________________________
bn_1 (BatchNormalization)    (None, 52, 52, 32)        128       
_________________________________________________________________
activation_1 (Activation)    (None, 52, 52, 32)        0         
_________________________________________________________________
mp_1 (MaxPooling2D)          (None, 26, 26, 32)        0         
__________

In [49]:
loaded_model.layers.pop()
print(loaded_model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100, 100, 3)       0         
_________________________________________________________________
bn_0 (BatchNormalization)    (None, 100, 100, 3)       12        
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 108, 108, 3)       0         
_________________________________________________________________
conv_1 (Conv2D)              (None, 52, 52, 32)        2432      
_________________________________________________________________
bn_1 (BatchNormalization)    (None, 52, 52, 32)        128       
_________________________________________________________________
activation_1 (Activation)    (None, 52, 52, 32)        0         
_________________________________________________________________
mp_1 (MaxPooling2D)          (None, 26, 26, 32)        0         
__________

In [29]:
X = loaded_model.layers[-1].output
X = Dense(3, activation = 'softmax', name = 'output_layer')(X)
model1 = Model(inputs = loaded_model.layers[0].output, outputs = X, name = 'next_model')
print(model1.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100, 100, 3)       0         
_________________________________________________________________
bn_0 (BatchNormalization)    (None, 100, 100, 3)       12        
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 108, 108, 3)       0         
_________________________________________________________________
conv_1 (Conv2D)              (None, 52, 52, 32)        2432      
_________________________________________________________________
bn_1 (BatchNormalization)    (None, 52, 52, 32)        128       
_________________________________________________________________
activation_1 (Activation)    (None, 52, 52, 32)        0         
_________________________________________________________________
mp_1 (MaxPooling2D)          (None, 26, 26, 32)        0         
__________

In [30]:
model1.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [38]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                  rotation_range = 20,
                                  width_shift_range = 0.2,
                                  height_shift_range = 0.2,
                                  horizontal_flip = True,
                                  fill_mode = 'nearest')
validation_datagen = ImageDataGenerator(rescale = 1./255)

train_batchsize = 64
validation_batchsize = 8

train_generator = train_datagen.flow_from_directory('Transfer_Train',
                                 target_size = (100, 100),
                                 batch_size = train_batchsize,
                                 class_mode = 'categorical',
                                 save_to_dir = 'blank_train')

validation_generator = validation_datagen.flow_from_directory('Transfer_vali',
                                 target_size = (100, 100),
                                 batch_size = validation_batchsize,
                                 class_mode = 'categorical',
                                 shuffle = False,
                                 save_to_dir = 'blank_test')

Found 297 images belonging to 3 classes.
Found 89 images belonging to 3 classes.


In [39]:
history = model1.fit_generator(train_generator,
                              steps_per_epoch=train_generator.samples/train_generator.batch_size,
                              epochs = 30,
                              validation_data = validation_generator,
                              validation_steps=validation_generator.samples/validation_generator.batch_size,
                              verbose = 1)

Epoch 1/30
5/4 [================================] - 10s 2s/step - loss: 1.3252 - acc: 0.3180 - val_loss: 1.0986 - val_acc: 0.3371
Epoch 2/30
5/4 [================================] - 9s 2s/step - loss: 1.1274 - acc: 0.3197 - val_loss: 1.1018 - val_acc: 0.3258
Epoch 3/30
5/4 [================================] - 9s 2s/step - loss: 1.0989 - acc: 0.3414 - val_loss: 1.1019 - val_acc: 0.3258
Epoch 4/30
5/4 [================================] - 12s 2s/step - loss: 1.1027 - acc: 0.3229 - val_loss: 1.1019 - val_acc: 0.3258
Epoch 5/30
5/4 [================================] - 5s 985ms/step - loss: 1.1014 - acc: 0.3275 - val_loss: 1.1018 - val_acc: 0.3258
Epoch 6/30
5/4 [================================] - 10s 2s/step - loss: 1.1029 - acc: 0.3156 - val_loss: 1.1018 - val_acc: 0.3258
Epoch 7/30
5/4 [================================] - 7s 1s/step - loss: 1.1023 - acc: 0.3246 - val_loss: 1.1017 - val_acc: 0.3258
Epoch 8/30
5/4 [================================] - 11s 2s/step - loss: 1.1006 - acc: 0.334